In [1]:
import numpy as np
import tensorflow as tf
import glob
import uproot
import uproot_methods

In [2]:
class AutoEncoder:
    def __init__(self, layer_sizes, learning_rate=0.01, num_iters=5000, init_weight_spread=1.0, batch_size=None):

        self.layer_sizes = layer_sizes
        self.learning_rate = 0.01
        self.num_iters = num_iters
        self.batch_size = batch_size

        self.X = tf.placeholder("float", [None, layer_sizes[0]])

        self.weights = {}
        self.biases = {}

        self.initializeWeights(init_weight_spread)
        self.encoder = self.getEncoder(self.X)
        self.decoder = self.getDecoder(self.encoder)
                
    def initializeWeights(self, weight_spread):
        for i in range(len(self.layer_sizes)-1):
            a = 4.0*np.sqrt(6.0/(self.layer_sizes[i] + self.layer_sizes[i-1]))
            self.weights["encoder_"+str(i)] = tf.Variable(tf.random_uniform([self.layer_sizes[i], self.layer_sizes[i+1]], -a, a))
            a = 4.0*np.sqrt(6.0/(self.layer_sizes[-i-1] + self.layer_sizes[-i-2]))
            self.weights["decoder_"+str(i)] = tf.Variable(tf.random_uniform([self.layer_sizes[-i-1], self.layer_sizes[-i-2]], -a, a))
        for i in range(len(self.layer_sizes)-1):
            self.biases["encoder_"+str(i)] = tf.Variable(tf.zeros([self.layer_sizes[i+1]]))
            self.biases["decoder_"+str(i)] = tf.Variable(tf.zeros([self.layer_sizes[-i-2]]))

    def getEncoder(self, x):
        ls = [x]
        for i in range(len(self.layer_sizes)-1):
            ls.append(tf.nn.sigmoid(tf.add(tf.matmul(ls[i], self.weights['encoder_'+str(i)]),
                                           self.biases['encoder_'+str(i)])))
        return ls[-1]
        
    def getDecoder(self, x):
        ls = [x]
        for i in range(len(self.layer_sizes)-1):
            ls.append(tf.nn.sigmoid(tf.add(tf.matmul(ls[i], self.weights['decoder_'+str(i)]),
                                           self.biases['decoder_'+str(i)])))
        return ls[-1]

    def train(self, sess, X, batch_size, output_every=None):
        self.y_pred = self.decoder
        self.y_true = self.X
        self.loss = tf.reduce_mean(tf.pow(self.y_true - self.y_pred, 2))
        self.optimizer = tf.train.RMSPropOptimizer(self.learning_rate).minimize(self.loss)

        sess.run(tf.global_variables_initializer())

        for i in range(1, self.num_iters+1):
            if self.batch_size is None:
                batch_X = X
            else:
                batch_X = X[np.random.choice(np.arange(X.shape[0]), batch_size, replace=False), :]

            _, l = sess.run([self.optimizer, self.loss], feed_dict={self.X:batch_X})

            if output_every is not None and (i==1 or i%output_every==0):
                print('Step {0}: Batch loss: {1}'.format(i, l))

        return l

    def run(self, sess, X, justEncoder=False):
        if justEncoder:
            res = sess.run(self.encoder, feed_dict={self.X: X})
        else:
            res = sess.run(self.decoder, feed_dict={self.X: X})
        return res

## Data Query and Preprocessing

In [4]:
#HistogramIntegral returns the total number of events
def HistogramIntegral(hist):
    return sum(hist[0][i] for i in range(len(hist[0])))

In [5]:
runs = []
hists = []
year = 2018
#norm_cut = None
norm_cut = 10000
max_bins = None
title = None
lumi_json = None
plot = "Segments/hSTimeCombined"
dname = plot.split('/')[0]
hname = plot.split('/')[1]
hpath = "DQMData/Run {}/CSC/Run summary/CSCOfflineMonitor"

In [6]:
if year ==2018:
        fnamesD =glob.glob("/eos/cms/store/group/comm_dqm/DQMGUI_data/Run2018/SingleMuon/*/DQM_V0001_R000*__SingleMuon__Run2018D-PromptReco-v2__DQMIO.root")
        fnamesABC = glob.glob("/eos/cms/store/group/comm_dqm/DQMGUI_data/Run2018/SingleMuon/*/DQM_V0001_R000*__SingleMuon__Run2018*-17Sep2018-*__DQMIO.root")
        fnames= fnamesD +fnamesABC
        
if year ==2017:
    fnames = []
    fnames =glob.glob("/eos/cms/store/group/comm_dqm/DQMGUI_data/Run2017/SingleMuon/*/DQM_V0001_R000*__SingleMuon__Run2017*-17Nov2017-v1__DQMIO.root")
    
if year ==2016:
    fnames = glob.glob("/eos/cms/store/group/comm_dqm/DQMGUI_data/Run2016/SingleMuon/*/DQM_V0001_R000*__SingleMuon__Run2016*-21Feb2020_UL2016_HIPM-v1__DQMIO.root")

In [ ]:
for fname in fnames:
    run = int(fname.split("/")[-1].split("__")[0][-6:])
    #Corrupted file
    if run == 315267:
        continue
    f = uproot.open(fname)
    #Fetch all the 1D histograms into a list
    histograms =f[hpath.format(run)].allitems(filterclass=lambda cls: issubclass(cls, uproot_methods.classes.TH1.Methods))
    for name, roothist in histograms:
        name = name.decode("utf-8")
        name = name.replace(";1", "")
        #Grab the 1D histogram we want
        if plot == name: 
            h = roothist.numpy()
            #Include only histograms that have enough events
            if norm_cut is None or HistogramIntegral(h) >= norm_cut:
                if max_bins==None:
                    nbins = len(h[0])
                else:
                    nbins = min(len(h[0]), max_bins)
                hists.append(h[0])
                runs.append(run)